# Example tracking workflow
## Comments
- Mention that throughout:
    - "coords" will be the full table of coordinates, along with assigned track IDs
    - "tracks" will be the coordinates of the most recent track instance
    - "points" will be the coordinates for the current frame

## Ideas
- Cover both Munkres (dense cost matrix) and JV LAPMOD (sparse cost matrix) for linking, comparing speed and memory allocation for different number of points.  This may be one for the talk, rather than having others do it.  That way it doesn't matter what memory is available.
- Start by getting a basic (adjacent frames only, no track splitting/merging) workflow completed.  For the previous frames, set start_frame and end_frame to the same value (i.e. frame-1).
- Add more factors to cost matrix.  The easiest to show may be conservation of size.  Mention that we could also add a threshold for size change, but we won't.  Example data will need to have instances of miss-linking that are resolved with size-based linking.
- Add multi-frame tracking by extending linking to n previous frames.  This will require addition of extraction of only the most recent instances of each track.


## Installing packages

In [ ]:
!pip install --user matplotlib
!pip install --user pandas
!pip install --user Pillow
!pip install --user scikit-image

## Importing libraries

In [1]:
import math
import sys
import time
import util

import numpy as np

from scipy.optimize import linear_sum_assignment

## Getting coordinates present in frame

In [2]:
def get_current_coords(coords, frame):
    # Identifying rows of "coords" with current frame number
    rows = coords.index[coords['T'] == frame]

    return rows

## Get available tracks

In [3]:
def get_available_tracks(coords, start_frame, end_frame):
    # Getting all rows present within the specified time interval
    rows = coords.index[(coords['T'] >= start_frame) & (coords['T'] <= end_frame)]
    
    # Getting the most recent instance of this track
    track_rows = []
    unique_IDs = np.unique(coords['TRACK_ID'][rows])
    for unique_ID in unique_IDs:
        if unique_ID != 0:
            # Getting rows for all instances of this track
            instances = coords.index[coords['TRACK_ID'] == unique_ID]    
            
            # Getting most recent instance of this track and appending it to final_rows
            track_rows.append(instances[-1:][0])
                    
    return track_rows

## Initialise first timepoint
All cells detected in the first timepoint can be considered "new" tracks.  As such, we give them each a unique track ID number.

In [4]:
def initialise_first_timepoint(coords):
    # Adding a blank extra column for the track ID
    coords['TRACK_ID'] = 0

    # Getting the row indices for the first frame
    idx_first = coords['T'] == 0

    coords.loc[idx_first,'TRACK_ID'] = range(1,sum(idx_first)+1)

## Calculate linking costs

In [5]:
def calculate_dense_cost_matrix(coords,track_rows,point_rows,thresh):
    # Creating the empty array
    costs = np.empty((len(track_rows),len(point_rows)))
        
    # Iterating over each pair, calculating the cost
    for track_i, track_row in enumerate(track_rows):
        for point_i,point_row in enumerate(point_rows):            
            cost = calculate_cost(coords,track_row,point_row,thresh)
            costs[track_i,point_i] = cost
           
    return costs
    
def calculate_cost(coords,track_row,point_row,thresh):
    # Spatial linking (distance between two points)
    dx = coords['X'][point_row] - coords['X'][track_row]
    dy = coords['Y'][point_row] - coords['Y'][track_row]
    d = math.sqrt(dx*dx + dy*dy)
    
    # If the two points are separated by more than the linking threshold, set them to infinity ('inf')
    if d > thresh:
        d = 1000000000
    
    return d

def calculate_cost_with_size(coords,track_row,point_row,thresh):
    # Spatial linking (distance between two points)
    dx = coords['X'][point_row] - coords['X'][track_row]
    dy = coords['Y'][point_row] - coords['Y'][track_row]
    d = math.sqrt(dx*dx + dy*dy)
    
    # ADD SIZE COST
    
    # If the two points are separated by more than the linking threshold, set them to infinity ('inf')
    if d > thresh:
        d = 1000000000
    
    return d

# Inheriting track IDs from previous frame
If the linked point in the previous frame already has an ID assigned to it, pass this on to the linked point (this SHOULD always be the case).  If our point wasn't assigned a link, set its track ID to the smallest unused value.

In [6]:
def assign_IDs(costs, assignments, coords, track_rows, point_rows):
    for track_assignment, point_assignment in zip(assignments[0],assignments[1]):
        # Even though we set disallowed costs to 1000000000, if there are no better options, Munkres will still assign this. 
        # We need to check if the assignment corresponds to one of these disallowed assignments.
        if costs[track_assignment,point_assignment] < 1000000000:
            ID = coords.at[track_rows[track_assignment],'TRACK_ID']
            coords.at[point_rows[point_assignment],'TRACK_ID'] = ID


# Assigning track IDs to unlinked points

In [7]:
def assign_new_IDs(coords, point_rows):
    # Getting the maximum track ID present in coords
    max_ID = coords.TRACK_ID.max()
    
    # Iterating over all current points and assigning the next available ID if they're still 0
    for point_row in point_rows:
        if coords.at[point_row,'TRACK_ID'] == 0:
            max_ID = max_ID + 1
            coords.at[point_row,'TRACK_ID'] = max_ID

## Main workflow

In [8]:
%%html
<style>
.output_wrapper button.btn.btn-default,
.output_wrapper .ui-dialog-titlebar {
  display: none;
}
</style>

In [9]:
%matplotlib notebook

# Setting parameters
np.set_printoptions(precision=3,threshold=sys.maxsize)
linking_thresh = 10
frame_thresh = 2

# # Loading image stack
path = "..\\data\\ExampleTimeseries.tif"
images = util.load_images(path);

# Loading coordinates
path = "../data/ObjectCoordinatesNoHeader.csv"
coords = util.load_coordinates(path);

# Getting the number of frames
n_frames = images.shape[2]

# Set new track IDs for each object in the first frame
initialise_first_timepoint(coords)

# Starting at frame 2, looping over each frame, linking pairs
print("")
for frame in range (1,n_frames):    
    sys.stdout.write("\rProcessing frame %i" % frame)
    
    start_frame = frame-1-frame_thresh
    end_frame = frame-1
    
    # Get row labels for the previous and current frame
    track_rows = get_available_tracks(coords,start_frame,end_frame)
    point_rows = get_current_coords(coords,frame)
    
    # Calculate costs for each possible link, then use Munkres to determine assignments
    costs = calculate_dense_cost_matrix(coords,track_rows,point_rows,linking_thresh)    
    assignments = linear_sum_assignment(costs)
        
    # Assigning links
    assign_IDs(costs,assignments,coords,track_rows,point_rows)
    assign_new_IDs(coords,point_rows)
    
print("")
print("Complete!")

Loading images from " ..\data\ExampleTimeseries.tif "
Loaded image shape:  (92, 700, 1100)
Reordered image shape:  (700, 1100, 92)
 
Loading coordinates from " ../data/ObjectCoordinatesNoHeader.csv "
Loaded data shape:  (8639, 6)
 

Processing frame 1
Processing frame 2
Processing frame 3
Processing frame 4
Processing frame 5
Processing frame 6
Processing frame 7
Processing frame 8
Processing frame 9
Processing frame 10
Processing frame 11
Processing frame 12
Processing frame 13
Processing frame 14
Processing frame 15
Processing frame 16
Processing frame 17
Processing frame 18
Processing frame 19
Processing frame 20
Processing frame 21
Processing frame 22
Processing frame 23
Processing frame 24
Processing frame 25
Processing frame 26
Processing frame 27
Processing frame 28
Processing frame 29
Processing frame 30
Processing frame 31
Processing frame 32
Processing frame 33
Processing frame 34
Processing frame 35
Processing frame 36
Processing frame 37
Processing frame 38
Processing frame

In [ ]:
# Adding track renders
util.show_overlay(images,coords)

Rendering frame 8 of 92